In [4]:
# Cell 1
# Dependency bootstrapper

import importlib
import subprocess
import sys
from pathlib import Path

VENDOR_DIR = Path.cwd() / '.ptse_vendor'
VENDOR_DIR.mkdir(parents=True, exist_ok=True)
if str(VENDOR_DIR) not in sys.path:
    sys.path.insert(0, str(VENDOR_DIR))


def _clear_from_modules(module_name: str) -> None:
    keys_to_drop = [key for key in sys.modules if key == module_name or key.startswith(f"{module_name}.")]
    for key in keys_to_drop:
        sys.modules.pop(key, None)


def ensure_package(module_name: str, pip_requirement: str) -> None:
    """Install module into the local vendor directory when import fails."""
    try:
        importlib.import_module(module_name)
        return
    except Exception:
        _clear_from_modules(module_name)

    command = [
        sys.executable,
        '-m',
        'pip',
        'install',
        '--upgrade',
        '--no-cache-dir',
        '--target',
        str(VENDOR_DIR),
        pip_requirement,
    ]
    subprocess.check_call(command)
    importlib.invalidate_caches()
    importlib.import_module(module_name)


ensure_package('pandas', 'pandas==2.3.3')
ensure_package('aequilibrae', 'aequilibrae')


No pre-existing parameter file exists for this project. Will use default


In [5]:
# Cell 2
# Project bootstrap helpers

import importlib
import sqlite3
import shutil
import subprocess
import sys
from pathlib import Path


def ensure_sqlite_extensions_ready() -> None:
    """Swap in pysqlite3 when the default sqlite build cannot load extensions."""
    if hasattr(sqlite3.Connection, 'enable_load_extension'):
        return

    try:
        import pysqlite3 as sqlite3_mod
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pysqlite3-binary'])
        import pysqlite3 as sqlite3_mod

    sys.modules['sqlite3'] = sqlite3_mod
    sys.modules['_sqlite3'] = sqlite3_mod
    importlib.invalidate_caches()


ensure_sqlite_extensions_ready()

try:
    import aequilibrae
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'aequilibrae'])
    import aequilibrae

from aequilibrae import Project
from importlib import metadata

try:
    aequilibrae_version = metadata.version('aequilibrae')
except metadata.PackageNotFoundError:
    aequilibrae_version = getattr(aequilibrae, '__version__', 'unknown')



def ensure_project_ready(project_path: Path) -> Project:
    """Create or open an Aequilibrae project at the requested path."""
    project_path = project_path.resolve()
    project_path.parent.mkdir(parents=True, exist_ok=True)
    project_db = project_path / 'project_database.sqlite'

    def _reset_project_folder() -> None:
        if project_path.exists():
            shutil.rmtree(project_path)

    def _project_has_minimum_tables(proj: Project) -> bool:
        try:
            with proj.db_connection as conn:
                return bool(
                    conn.execute("SELECT 1 FROM sqlite_master WHERE name='modes'").fetchone()
                )
        except Exception:
            return False

    project = Project()
    if project_db.exists():
        try:
            project.open(project_path.as_posix())
            if not _project_has_minimum_tables(project):
                raise RuntimeError('Incomplete Aequilibrae database detected.')
        except Exception:
            project.close()
            _reset_project_folder()
            project = Project()
            project.new(project_path.as_posix())
            project.open(project_path.as_posix())
    else:
        if project_path.exists():
            leftover_items = list(project_path.iterdir())
            if leftover_items:
                raise FileExistsError(
                    'Project folder exists without a database. Remove its contents or choose another path.'
                )
            project_path.rmdir()
        project.new(project_path.as_posix())
        project.open(project_path.as_posix())
    return project


print(f'aequilibrae version: {aequilibrae_version}')


aequilibrae version: 1.5.0


In [6]:
# Cell 3
# Project health check

workspace_path = Path('data') / 'ptse_project'
project = ensure_project_ready(workspace_path)
print(f'Project prepared at: {workspace_path.resolve()}')

with project.db_connection as conn:
    tables = [
        row[0]
        for row in conn.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name").fetchall()
    ]

if tables:
    print('Tables available in the project database:')
    for table_name in tables:
        print(f'- {table_name}')
else:
    print('No tables were found in the project database yet.')


Project prepared at: C:\Users\Mongkut\Desktop\PTSE\PTSE\data\ptse_project
Tables available in the project database:
- ElementaryGeometries
- KNN
- SpatialIndex
- about
- attributes_documentation
- data_licenses
- geometry_columns
- geometry_columns_auth
- geometry_columns_field_infos
- geometry_columns_statistics
- geometry_columns_time
- idx_links_geometry
- idx_links_geometry_node
- idx_links_geometry_parent
- idx_links_geometry_rowid
- idx_nodes_geometry
- idx_nodes_geometry_node
- idx_nodes_geometry_parent
- idx_nodes_geometry_rowid
- idx_zones_geometry
- idx_zones_geometry_node
- idx_zones_geometry_parent
- idx_zones_geometry_rowid
- link_types
- links
- matrices
- migrations
- modes
- nodes
- periods
- results
- scenarios
- spatial_ref_sys
- spatial_ref_sys_aux
- spatialite_history
- sql_statements_log
- sqlite_sequence
- transit_graph_configs
- views_geometry_columns
- views_geometry_columns_auth
- views_geometry_columns_field_infos
- views_geometry_columns_statistics
- virts_ge